## Useful Group-By Operations

This code should help speed up your data exploration for any new dataset

In [36]:
import pandas as pd
import seaborn as sns; sns.set()
import numpy as np
tips = sns.load_dataset("tips")

# Allows multiple output from each cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


### Basic Groupby Operations

In [5]:
# Finding the numbers of each group
tips.groupby(['sex']).size()

tips.groupby(['time']).size()

sex
Male      157
Female     87
dtype: int64

time
Lunch      68
Dinner    176
dtype: int64

In [27]:
# Basic operation for a group. 
# Taking the mean of a group by will return all continuous features
tips.groupby(['sex']).mean()

,total_bill,tip,size
sex,,,
Male,20.744076,3.089618,2.630573
Female,18.056897,2.833448,2.459770


In [28]:
# The mode of groups. You'll get some meaningless results from continuous features, but the categorical ones are of value.
tips.groupby(['sex']).agg(lambda x:x.value_counts().index[0])

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Male,21.01,2.0,No,Sat,Dinner,2
Female,13.00,2.0,No,Thur,Dinner,2


#### Easily extend to multiple groups

In [20]:
tips.groupby(['sex', 'time']).mean()

total_bill       tip      size
sex    time                                  
Male   Lunch    18.048485  2.882121  2.363636
       Dinner   21.461452  3.144839  2.701613
Female Lunch    16.339143  2.582857  2.457143
       Dinner   19.213077  3.002115  2.461538

In [22]:
tips.groupby(['sex', 'time', 'smoker']).mean()

total_bill       tip      size
sex    time   smoker                                
Male   Lunch  Yes      17.374615  2.790769  2.153846
              No       18.486500  2.941500  2.500000
       Dinner Yes      23.642553  3.123191  2.595745
              No       20.130130  3.158052  2.766234
Female Lunch  Yes      17.431000  2.891000  2.300000
              No       15.902400  2.459600  2.520000
       Dinner Yes      18.215652  2.949130  2.217391
              No       20.004138  3.044138  2.655172

## More complicated group by aggregations

Using .agg, we can get more specific information from groups

In [24]:
# Getting mean from a single feature for a group
tips.groupby(['smoker']).agg({'total_bill':'mean'})

,total_bill
smoker,
Yes,20.756344
No,19.188278


In [25]:
# Getting multiple aggregations
tips.groupby(['smoker']).agg({'total_bill':['mean', 'max']})

total_bill       
             mean    max
smoker                  
Yes     20.756344  50.81
No      19.188278  48.33

In [29]:
# Extend to multiple groups
tips.groupby(['smoker', 'sex']).agg({'total_bill':['mean', 'max', 'min']})

total_bill             
                    mean    max   min
smoker sex                           
Yes    Male    22.284500  50.81  7.25
       Female  17.977879  44.30  3.07
No     Male    19.791237  48.33  7.51
       Female  18.105185  35.83  7.25

In [30]:
# Adding multiple groups with multiple features
tips.groupby(['smoker', 'sex']).agg({'total_bill':['mean', 'max', 'min'], 'tip':['mean', 'max']})

total_bill                    tip      
                    mean    max   min      mean   max
smoker sex                                           
Yes    Male    22.284500  50.81  7.25  3.051167  10.0
       Female  17.977879  44.30  3.07  2.931515   6.5
No     Male    19.791237  48.33  7.51  3.113402   9.0
       Female  18.105185  35.83  7.25  2.773519   5.2

#### Functions within group-by

We can define our own functions if there are specific interests, such as understanding more about the distribution

In [40]:
### Define a function to find percentiles
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [56]:
tips.groupby(['sex']).agg({'total_bill':[percentile(25), percentile(50), percentile(75), 'mean', 'std']})

total_bill                                                 
       percentile_25 percentile_50 percentile_75       mean       std
sex                                                                  
Male           14.00         18.35         24.71  20.744076  9.246469
Female         12.75         16.40         21.52  18.056897  8.009209

This now gives a much better picture of the overall distribution. We have the mean, standard deviation, as well as the 25th, 50th, and 75th percentile. It appears that men have larger bills overall, but have a wider spread. In a typical data exploration phase, we might start to look more at the tips. But this would be a great start for understanding your data

### Collapsing the index for group by dataframe

The indicies for groupby dataframes are annoying to work with. You can collapse them with the following code

In [60]:
df = tips.groupby(['sex']).agg({'total_bill':[percentile(25), percentile(75), 'mean', 'std']})
df

total_bill                                   
       percentile_25 percentile_75       mean       std
sex                                                    
Male           14.00         24.71  20.744076  9.246469
Female         12.75         21.52  18.056897  8.009209

In [61]:
df.columns = ['_'.join(col).strip() for col in df.columns.values]

In [62]:
df

,total_bill_percentile_25,total_bill_percentile_75,total_bill_mean,total_bill_std
sex,,,,
Male,14.00,24.71,20.744076,9.246469
Female,12.75,21.52,18.056897,8.009209


#### With multiple groups and aggregations

In [68]:
df = (tips.groupby(['sex', 'smoker']).agg({'total_bill':[percentile(25), percentile(75), 'mean', 'std'], 
                                          'tip':[percentile(25), percentile(75), 'mean', 'std']}))
df

total_bill                                              tip  \
              percentile_25 percentile_75       mean       std percentile_25   
sex    smoker                                                                  
Male   Yes          15.2725       28.5725  22.284500  9.911845           2.0   
       No           13.8100       22.8200  19.791237  8.726566           2.0   
Female Yes          12.7600       22.1200  17.977879  9.189751           2.0   
       No           12.6500       20.8625  18.105185  7.286455           2.0   

                                                 
              percentile_75      mean       std  
sex    smoker                                    
Male   Yes           3.8200  3.051167  1.500120  
       No            3.7100  3.113402  1.489559  
Female Yes           3.5000  2.931515  1.219916  
       No            3.4375  2.773519  1.128425

In [69]:
df.columns = ['_'.join(col).strip() for col in df.columns.values]
df

total_bill_percentile_25  total_bill_percentile_75  \
sex    smoker                                                       
Male   Yes                      15.2725                   28.5725   
       No                       13.8100                   22.8200   
Female Yes                      12.7600                   22.1200   
       No                       12.6500                   20.8625   

               total_bill_mean  total_bill_std  tip_percentile_25  \
sex    smoker                                                       
Male   Yes           22.284500        9.911845                2.0   
       No            19.791237        8.726566                2.0   
Female Yes           17.977879        9.189751                2.0   
       No            18.105185        7.286455                2.0   

               tip_percentile_75  tip_mean   tip_std  
sex    smoker                                         
Male   Yes                3.8200  3.051167  1.500120  
       No                 3.7100  3.113402  1.489559  
Female Yes                3.5000  2.931515  1.219916  
       No                 3.4375  2.773519  1.128425

##### Now the dataframe is much easier to work with